# STA 141B Data & Web Technologies for Data Analysis

### Lecture 13, 2/19/24, Principal Component Analysis


### Announcements

- Homework 3 due this Friday

### Last week's topics

- Tagging
- Latent Dirichlet Allocation

### Today's topics
- Principal Component Analysis
    - Computation
    - Choice of k
    - Approximation of eigenvalues
    

### Recap

In [1]:
import numpy as np

#### [&#128011;](https://www.gutenberg.org/files/2701/2701-h/2701-h.htm)

In [2]:
import nltk
import re

In [254]:
moby = nltk.corpus.gutenberg.raw("melville-moby_dick.txt")
pattern = r"(?<!,\s{1})(?:ETYMOLOGY|CHAPTER\s{1}\d+|Epilogue|EXTRACTS(?=\s*\())(?:\.*\s*).+?\s*.*[\.{1}|!{1}|?{1}|\){1}]"
corpus = re.split(pattern, moby)
corpus.pop(0)
corpus = [re.sub(r"\s+", " ", document).lower() for document in corpus]

In [255]:
cetological = [0, 1, 46, 54, 59, 60, 61, 63, 64, 68, 75, 76, 77, 78, 80, 81, 85, 86, 87, 89, 90, 91, 93, 96, 97, 98, 102, 103, 104, 105, 106, 107]
story = [i not in cetological for i in range(138)]
labels = [i for i in map(lambda x: 'story' if x else 'ceto', story)]

In [256]:
corpus_tokenized = [nltk.word_tokenize(document) for document in corpus]

In [257]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend([',', '.', ':', '!', ';', '?', '--', '\'', '\'\'', '\'s', '``'])

In [258]:
corpus_tokenized = [[word for word in document if word not in stopwords] for document in corpus_tokenized] # remove stopwords
corpus_tokenized = [[nltk.PorterStemmer().stem(word) for word in document] for document in corpus_tokenized]

In [259]:
dictionary = [d for c in corpus_tokenized for d in c]
fq = nltk.FreqDist(dictionary)
fq = {k: v for k, v in dict(fq).items() if v > 20}
corpus_tokenized = [[word for word in document if word in fq] for document in corpus_tokenized] # remove stopwords

In [260]:
corpus = [' '.join(document) for document in corpus_tokenized]

In [261]:
from sklearn.feature_extraction.text import CountVectorizer

In [262]:
vec = CountVectorizer(tokenizer = nltk.word_tokenize)
freq = vec.fit_transform(corpus)
corpus_freq = freq.todense() # 

In [263]:
corpus_freq = np.array(corpus_freq) # removes warning further down

In [264]:
from sklearn.decomposition import LatentDirichletAllocation

In [290]:
ntopics = 3
lda = LatentDirichletAllocation(n_components = ntopics, 
                                learning_method = 'online', 
                                random_state = 2024)
lda.fit(corpus_freq)
posterior = lda.transform(corpus_freq)

In [291]:
import pandas as pd

In [292]:
df = pd.DataFrame(posterior).reset_index()
df.columns = ['chapter'] + ['Topic ' + str(i + 1) for i in range(0,ntopics)]

In [293]:
ddf = pd.melt(df, id_vars = 'chapter')

In [294]:
import plotly.express as px
fig = px.line(ddf, x="chapter", y="value", color='variable', labels={
                     "chapter": "Chapter",
                     "value": "Probability",
                     "variable": "LDA Topics"
                 }, title = 'LDA Probabilities: Labelled cetology chapters are shaded')

for i, e in enumerate(labels): 
    if e=='ceto': 
        fig.add_vrect(x0=i-0.5, x1=i+0.5, line_width=0, fillcolor="grey", opacity=0.2)

fig.show()

### Computation

Consider a data set $X\in\mathbb{R}^{n\times p}$ for $n, p\in\mathbb{N}$. 
Each observation corresponds to a random vector $x = (x_1, \dots , x_p)'$  with known expectation $E(x) = \mu\in\mathbb{R}^p$ and covariance $Cov(x) = \Sigma = \mathbb{R}^{p\times p}$. 

Its principal components are linear combinations of $x_1, \dots , x_p$. Specifically, we aim to find a tranformation of $x$ so that the covariance of $U(x-\mu)$, $U\in\mathbb{R}^{p\times k}$, $k\leq p$ has a simple structure and retains as much information about $\Sigma$ as possible. 

Let $u_1\in\mathbb{R}^p$ so that $Cov(u_1'(x-\mu)) =  Cov(u_1'x) = u_1'\Sigma u_1$ is maximal. 
Since $u_1$ can be arbitrarily large, we limit $\|u_1\|^2 = 1$. 

To maximize $u_1'\Sigma u_1$ under this constraint, one can use Lagrange multiplication for $\lambda>1$.  
$$
\max u_1'\Sigma u_1 - \lambda(\|u_1\|^2 - 1). 
$$
Taking derivatives gives $(\Sigma - \lambda I_p) u_1 = 0$. Hence, $u_1$ is an eigenvector to eigenvalue $\lambda$ of $\Sigma$. 
Since $u_1'\Sigma u_1$ ought to be maximized, $\lambda$ should be as large as possible. 
Consequently, $u_1$ is an eigenvector to the largest eigenvalue $\lambda$ of $\Sigma$. 
The vector $u_1'(x-\mu)$ is the first *principal component* of $x$.

Generally, one can show that the $k$-th principal component is $u_k'(x-\mu)$ is the eigenvector corresponding to the $k$-th eigenvalue of $\Sigma$. 
The vector $u_k\in\mathbb{R}^p$ is the vectors of loadings of the $k$-th principal component. 

For $U\in\mathbb{R}^{p\times k}$ be an orthogonal matrix, which $k$-th column is $u_k$ and $z = U'(x − \mu)\in\mathbb{R}^k$, $k \leq p$ the vector, which $k$-th element is the $k$-th principal component. 

Note that for $k=p$ we have $$\Sigma = U\text{diag}(\lambda_1, \dots, \lambda_p)U'$$ and $$Cov(z) = E(zz') = U'E((x-\mu)(x-\mu)')U = U'\Sigma U = \text{diag}(\lambda_1, \dots, \lambda_p).$$ 

In practive, $\mu$ and $\Sigma$ are unkown. For the data set $X\in\mathbb{R}^{n\times p}$, the empirical principal components are given from the eigenvalue decomposition $\widehat\Sigma = \widehat U\text{diag}(\lambda_1, \dots, \lambda_p)\widehat U'$. 
If $n>p$ one may use the consistent estimators 
$$
\widehat\Sigma = \frac{1}{n - 1}\widetilde{X}'\widetilde{X}, 
$$
where $\widetilde{X}$ is the matrix with entries $\widetilde{X}_{ij} = x_{ij} - \bar{x}_j$, $\bar{x}_j = n^{-1}\sum_{i=1}^n x_{ij}$ is the estimator for $\mu_j$. 
This gives the scores $Z = \widetilde X \widehat{U}\in\mathbb{R}^{n\times p}$. 

In [306]:
X = corpus_freq
xbar = corpus_freq.mean(axis = 0)
Xtilde = X - xbar

In [307]:
Xtilde.mean(axis = 0)

array([ 2.37330284e-17, -9.73456420e-17,  1.29526020e-16, ...,
       -2.28480680e-16,  1.74980803e-17,  3.04104568e-16])

In [308]:
n = X.shape[0]
n

138

In [309]:
Sigmahat = Xtilde.T @ Xtilde / (n - 1)

In [310]:
Sigmahat.shape

(1029, 1029)

In [311]:
from numpy.linalg import eig

In [312]:
# e, v = eig(Sigmahat) # not possible, p is too large

In [313]:
e = [a for a, b in my_eigen(Sigmahat, 20)] # function below
e

[545.4954776146957,
 269.80228037200095,
 113.70462484246383,
 85.59963056731561,
 64.38395690685533,
 66.89261555725157,
 51.700633248366884,
 50.194564392205976,
 43.48801962589618,
 38.10420733383622,
 34.90287248996064,
 29.681102527288395,
 24.883572033858588,
 24.191951901564195,
 23.514605589642137,
 19.582619485553334,
 20.659676284176633,
 19.168494881885625,
 17.68034916596792,
 16.75781704353566]

### Choice of $k$

We want to use PCA as a dimension reduction. The $p$-dimensinal data is to be reduced to some $k$-dimensional data, $k\le p$, while preserving as much variation (information) as possible. Since

$$
Cov(z) = \text{diag}(\lambda_1, \dots, \lambda_p), 
$$

this corresponds on choosing the first $k$ empirical components. But how to choose $k$? 

In [314]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame({'eigen': e, 'index': range(20)})
fig = px.line(df, x="index", y="eigen")
fig.show()

The ellbow plot shows the relative variance - the number of PC to be chosen corresponds to the location in the plot, in which the additional variance explained per PC decreases. This is a subjective measure.  

### Computation of Eigenvalues

The computation of eigenvalues is complicated if $p$ is large. Here, we will learn about an approximate way to compute the $k$ largest eigenvectors and -values.  

Let $A\in\mathbb{R}^{p\times p}$ be a symmetric positiv semi-definite matrix, so that $A=V\mbox{diag}(\mu_1,\ldots,\mu_p)V^t$. 
The columns of $V=(v_1,\ldots,v_p)$ are the eigenvectors $v_i$ and $\mu_1>\mu_2>\ldots>\mu_p\geq 0$. To estimate $v_1$, start with any random vector $v^{(0)}$ and iterate 
$$
v^{(j+1)}_1=\frac{A^jv^{(0)}}{\|A^jv^{(0)}\|},\;\;j=1,2,\ldots
$$
until convergence. The corresponding eigenvalue is due to the Rayleigh ratio
$$
\mu_1=\frac{v_1^tAv_1}{v_1^tv_1},
$$
since 
$$
\frac{v_1^tAv_1}{v_1^tv_1}=\frac{(Av_1)^tv_1}{v_1^tv_1}=\frac{\mu_1 v_1^tv_1}{v_1^tv_1}=\mu_1.
$$
The second eigenvector can be retrieved via $A-\mu_1v_1v_1^t$ (there are other methods as well).

$$
v^{(k+1)}_1=\frac{A^kv^{(0)}}{\|A^kv^{(0)}\|},\;\;k=1,2,\ldots
$$

Why does this work? Since the eigenvectors are orthogonal, each vector $c\in\mathbb{R}^p$ can be represented as a linear combination of eigenvectors, i.e., $c=\sum_{i=1}^p c_iv_i$, $c_i\geq 0$ with $c_1>0$. Multiplicatoin on both sides with $A$ gives 
$$
Ac=\sum_{i=1}^pc_iAv_i=\sum_{i=1}^pc_i\mu_iv_i.
$$
$k$-times repetition gives 
$$
A^kc=\sum_{i=1}^pc_i\mu_i^kv_i=\mu_1^k\left\{c_1v_1+c_2v_2\left(\frac{\mu_2}{\mu_1}\right)^k+\ldots+c_pv_p\left(\frac{\mu_p}{\mu_1}\right)^k\right\}.
$$
For $\mu_1\gg\mu_2$, $A^kc$ converges to $v_1(c_1\mu_1^k)$ for growing $k$. This convergence can be very slow if $\mu_1\approx\mu_2$. 

In [315]:
from numpy.linalg import norm

In [316]:
def my_eigen(A, k = 10, n = 20):
    
    e = 0
    v = np.repeat(1, A.shape[0])
    

    result = [None] * k
    for i, r in enumerate(result): 
        A = A - e * np.matrix(v).T @ np.matrix(v)
        e, v = my_rayleigh(A, n)
        result[i] = (e, v)

    return result


def my_rayleigh(A, n): 
    v = np.repeat(1, A.shape[0])
    
    for _ in range(n): 
        v = A.dot(v)
        v = np.squeeze(np.asarray(v / norm(v)))

    e = v.dot(A).dot(v).item()
    
    return e, v

In [317]:
[a for a, b in my_eigen(Sigmahat, 4)]

[545.4954776146957, 269.80228037200095, 113.70462484246383, 85.59963056731561]

In [318]:
e = eig(Sigmahat)[0]
e = np.sort(e)[::-1]
e[:4]

array([545.49547761+0.j, 269.80228037+0.j, 113.95899227+0.j,
        85.2607562 +0.j])

### Concluding Remarks

- If the underlying matrix $X$ is sparse, centralizing will un-sparse it. This may be computationally prohibitive. Often, in such cases, an uncentered PCA is used. 

In [319]:
[a for a, b in my_eigen(X.T @ X / (n - 1), 4)]

[1220.567491973623, 277.7863828600998, 113.38487347999346, 87.40330367164265]

In [356]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

In [360]:
pca = TruncatedSVD(n_components=4)
pca.fit(X.T)

TruncatedSVD(n_components=4)

In [361]:
(pca.singular_values_)**2/(n-1) 

array([1220.56749197,  277.78638286,  113.96745618,   86.64258956])

In [362]:
pca = PCA(n_components=4)
pca.fit(X.T)
(pca.singular_values_)**2/(n-1) 

array([818.21166573, 264.68298309, 113.96449431,  83.64995196])

- The PC are not interpretable by themselves. 
- PCA can not reduce the dimensionality of observations on a low-dimensinal non-linear manifold